# Upgrading from MathProgBase.jl to MathOptInterface.jl

+ MathProgBase.jl has been [depreciated](https://github.com/JuliaOpt/MathProgBase.jl/pull/231) since Jan 2019. It is therefore not officially supported by julia v1.2 and above. We should use [MathOptInterface.jl](https://github.com/jump-dev/MathOptInterface.jl) instead. 
+ Other packages such as BinaryProvider.jl and Ipopt.jl needs to also be updated to the latest version or depreciated and use the latest one supported by the Julia community.
+ To ensure a smooth update, checkout the [official documentation](https://jump.dev/MathOptInterface.jl/dev/tutorials/mathprogbase/) as well as examples by [WiSER.jl](https://github.com/OpenMendel/WiSER.jl/blob/master/src/fit.jl) and [OrdinalMultinomialModels.jl](https://github.com/OpenMendel/OrdinalMultinomialModels.jl/blob/master/src/ordmnfit.jl)

In [1]:
using Revise
using Ipopt
using QuasiCopula
using Random
using GLM
using LinearAlgebra
using Statistics
using StatsBase
using MathOptInterface
using RCall

ENV["COLUMNS"] = 240
BLAS.set_num_threads(1)
Threads.nthreads()

Precompiling QuasiCopula
  ✓ QuasiCopula
  1 dependency successfully precompiled in 5 seconds. 151 already precompiled.


1

In [7]:
k = 0 # number of causal SNPs

qc_model, G, βtrue, θtrue, γtrue, τtrue = simulate_longitudinal_traits(
    n = 10000, # sample size
    d_min = 5, # min number of observations per sample
    d_max = 5, # max number of observations per sample
    p = 15, # number of fixed effects, including intercept
    m = 2, # number of variance components
    q = 1000, # number of SNPs
    k = k, # number of causal SNPs
    seed = 2024,
    y_distribution = NegativeBinomial,
    τtrue = 0.5,
    T = Float64,
    maf = 0.3,
    causal_snp_β = 0.2
)

@show qc_model;

@time optm = QuasiCopula.fit!(qc_model);

initializing β using GLM.jl
gcm.β = [1.071141333938132, -0.4820165636340641, 0.48187754351752365, 0.48479332423135835, 0.49028177661345046, -0.4862122196580561, 0.48011767932560134, -0.4909547903170821, 0.48343181680091807, -0.48629241376979127, -0.4819929966888139, -0.48739697346794014, 0.4847745221649089, -0.48215713945837935, -0.4820357318329954]
initializing variance components using MM-Algorithm
gcm.θ = [1.0592722715027482e-6, 6.122833003834026e-8]
initializing r using Newton update
qc_model = Quasi-Copula Variance Component Model
  * base distribution: NegativeBinomial
  * link function: LogLink
  * number of clusters: 10000
  * cluster size min, max: 5, 5
  * number of variance components: 2
  * number of fixed effects: 15

initializing β using GLM.jl
gcm.β = [1.071141333938132, -0.4820165636340641, 0.48187754351752365, 0.48479332423135835, 0.49028177661345046, -0.4862122196580561, 0.48011767932560134, -0.4909547903170821, 0.48343181680091807, -0.48629241376979127, -0.4819929966

In [8]:
@show βtrue
@show qc_model.β
@show qc_model.∇β

@show θtrue
@show qc_model.θ
@show qc_model.∇θ;

@show τtrue
@show qc_model.τ
@show qc_model.∇τ;

βtrue = [1.0, -0.5, 0.5, 0.5, 0.5, -0.5, 0.5, -0.5, 0.5, -0.5, -0.5, -0.5, 0.5, -0.5, -0.5]
qc_model.β = [1.0046063367642517, -0.4984887784364577, 0.4975106668356966, 0.49863364342686267, 0.5005362319435009, -0.49937749222382477, 0.49609860587371496, -0.5030691251178447, 0.4979088170319177, -0.498723512131695, -0.4979842200695971, -0.5024552810277305, 0.49939421584685334, -0.49742550023645493, -0.497896132492108]
qc_model.∇β = [2.513183864475003, -0.41891552611386906, 0.5076179438792808, 0.6513679102365761, 0.11192276642960897, -0.44161617922173546, 0.6976507602926296, -0.16788531383480954, 0.47155179945564774, -0.31983265013316986, -0.005135645341738437, -0.5107490477872996, 0.345653826623753, -0.37613945434467144, -0.5770318666538211]
θtrue = [0.1, 0.1]
qc_model.θ = [0.10011179785376477, 0.08230665974530209]
qc_model.∇θ = [6.606855579200783, 7.916574470502383]
τtrue = 0.5
qc_model.τ = [1.0]
qc_model.∇τ = [2.139998567e-314]


In [9]:
qc_model.r # rtrue = 10 (negative binomial only)

1-element Vector{Float64}:
 10.026333882009538

# AR

In [1]:
using Revise
using Ipopt
using QuasiCopula
using Random
using GLM
using LinearAlgebra
using Statistics
using StatsBase
using MathOptInterface
using RCall

Precompiling QuasiCopula
  ✓ QuasiCopula
  1 dependency successfully precompiled in 4 seconds. 151 already precompiled.


In [2]:
# get data
R"""
library("gcmr")
data("epilepsy", package = "gcmr")
"""
@rget epilepsy

# form model
df = epilepsy
y = :counts
grouping = :id
d = Normal() # Poisson()
link = IdentityLink() # LogLink()
covariates = [:visit, :trt]
qc_model = AR_model(df, y, grouping, covariates, d, link; penalized=true)

# fit
QuasiCopula.fit!(qc_model)

gcm.β = [31.41573849878938, -22.96610169491528, -0.339631336405546]
initializing dispersion using residual sum of squares
gcm.τ = [0.0037930980963653453]
initializing AR(1) noise paramter using MM-algorithm

******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit https://github.com/coin-or/Ipopt
******************************************************************************

This is Ipopt version 3.14.14, running with linear solver MUMPS 5.6.2.

Number of nonzeros in equality constraint Jacobian...:        0
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian.............:        0

Total number of variables............................:        6
                     variables with only lower bounds:        2
             

-1230.9645317304487

In [6]:
@show qc_model.β
@show qc_model.σ2
@show qc_model.τ
@show qc_model.ρ;

qc_model.β = [33.906723110174724, -22.966101624887735, -0.03905523987816367]
qc_model.σ2 = [0.2833334216154253]
qc_model.τ = [0.004425825590418177]
qc_model.ρ = [1.0000000099997475]


# CS

In [1]:
using Revise
using Ipopt
using QuasiCopula
using Random
using GLM
using LinearAlgebra
using Statistics
using StatsBase
using MathOptInterface
using RCall

Precompiling QuasiCopula
  ✓ QuasiCopula
  1 dependency successfully precompiled in 5 seconds. 151 already precompiled.


In [2]:
# get data
R"""
data(respiratory, package="geepack")
respiratory_df <- respiratory[order(respiratory$id),]
"""
@rget respiratory_df

# form model
df = respiratory_df
y = :outcome
grouping = :id
d = Normal() # Bernoulli()
link = IdentityLink() # LogitLink()
covariates = [:center, :age, :baseline]
qc_model = CS_model(df, y, grouping, covariates, d, link)

# fit
QuasiCopula.fit!(qc_model)

gcm.β = [0.31424644209305314, 0.13302522431248406, -0.0036949643843916823, 0.37371312255483236]
initializing dispersion using residual sum of squares
gcm.τ = [5.014377157801042]
initializing CS noise paramter using MM-algorithm

******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit https://github.com/coin-or/Ipopt
******************************************************************************

This is Ipopt version 3.14.14, running with linear solver MUMPS 5.6.2.

Number of nonzeros in equality constraint Jacobian...:        0
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian.............:        0

Total number of variables............................:        7
                     variables with only lower bounds:  

┌ Warning: Optimization unsuccesful; got ITERATION_LIMIT
└ @ QuasiCopula ~/.julia/dev/QuasiCopula/src/parameter_estimation/fit_gaussian_ar_cs.jl:65


-738.7017912598116

In [4]:
@show qc_model.β
@show qc_model.σ2
@show qc_model.ρ;

qc_model.β = [0.473432329115939, 1.1371817357118306, -0.03481901279902936, 0.7013433295574378]
qc_model.σ2 = [41475.14332109955]
qc_model.ρ = [0.6158813782222337]
